In [1]:
import heapq
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
import random

class TaskManager:
    def __init__(self):
        self.tasks = pd.DataFrame(columns=['description', 'priority'])
        self.vectorizer = CountVectorizer()
        self.clf = MultinomialNB()
        self.model = make_pipeline(self.vectorizer, self.clf)

        # Load pre-existing tasks from a CSV file (if any)
        try:
            self.tasks = pd.read_csv('tasks.csv')
            self.model.fit(self.tasks['description'], self.tasks['priority'])
        except FileNotFoundError:
            pass

    def save_tasks(self):
        self.tasks.to_csv('tasks.csv', index=False)

    def add_task(self, description, priority):
        new_task = pd.DataFrame({'description': [description], 'priority': [priority]})
        self.tasks = pd.concat([self.tasks, new_task], ignore_index=True)
        self.save_tasks()

    def remove_task(self, description):
        self.tasks = self.tasks[self.tasks['description'] != description]
        self.save_tasks()

    def list_tasks(self):
        if self.tasks.empty:
            print("No tasks available.")
        else:
            print(self.tasks)

    def prioritize_task(self, description, new_priority):
        for i, (_, task) in enumerate(self.tasks.iterrows()):
            if task['description'] == description:
                self.tasks.at[i, 'priority'] = new_priority
                self.save_tasks()
                self.model.fit(self.tasks['description'], self.tasks['priority'])
                return
        print(f"Task '{description}' not found.")

    def recommend_tasks(self, keyword):
        if not self.tasks.empty:
            matching_tasks = self.tasks[self.tasks['description'].str.contains(keyword, case=False)]

            if not matching_tasks.empty:
                 print(f"Recommended tasks with '{keyword}':")
                 for i, (_, task) in enumerate(matching_tasks.iterrows(), start=1):
                      print(f"{i}. {task['description']} - Priority: {task['priority']}")
            else:
                print(f"No tasks found with '{keyword}'.")
        else:
             print("No tasks available for recommendations.")


def main():
    task_manager = TaskManager()

    while True:
        print("\nTask Management App:")
        print("1. Add Task")
        print("2. Remove Task")
        print("3. List Tasks")
        print("4. Prioritize Task")
        print("5. Recommend Tasks")
        print("6. Exit")

        choice = input("Enter your choice (1-6): ")

        if choice == '1':
            description = input("Enter task description: ")
            priority = input("Enter task priority (Low/Medium/High): ").capitalize()
            task_manager.add_task(description, priority)
            print("Task added successfully.")
        elif choice == '2':
            description = input("Enter task description to remove: ")
            task_manager.remove_task(description)
            print("Task removed successfully.")
        elif choice == '3':
            task_manager.list_tasks()
        elif choice == '4':
            description = input("Enter task description to prioritize: ")
            new_priority = input("Enter new priority (Low/Medium/High): ").capitalize()
            task_manager.prioritize_task(description, new_priority)
            print("Task prioritized successfully.")
        elif choice == '5':
            keyword = input("Enter a keyword for task recommendations: ")
            task_manager.recommend_tasks(keyword)
        elif choice == '6':
            print("Exiting the Task Management App. Goodbye!")
            break
        else:
            print("Invalid choice. Please enter a number between 1 and 6.")

if __name__ == "__main__":
    main()


Task Management App:
1. Add Task
2. Remove Task
3. List Tasks
4. Prioritize Task
5. Recommend Tasks
6. Exit
Enter your choice (1-6): 1
Enter task description: bathing
Enter task priority (Low/Medium/High): high
Task added successfully.

Task Management App:
1. Add Task
2. Remove Task
3. List Tasks
4. Prioritize Task
5. Recommend Tasks
6. Exit
Enter your choice (1-6): 3
  description priority
0     bathing     High

Task Management App:
1. Add Task
2. Remove Task
3. List Tasks
4. Prioritize Task
5. Recommend Tasks
6. Exit
Enter your choice (1-6): 6
Exiting the Task Management App. Goodbye!
